In [3]:
# import pandas as pd
# df = pd.read_csv('02_fileNameAndDocUrl.csv')
# df

,application_number,file_name,url
0,2025/298,Decision Notice - Decision notice 2025-298,https://plandocs.tandridge.gov.uk/my-requests/...
1,2025/298,Officer Report - Officer report-2025-298,https://plandocs.tandridge.gov.uk/my-requests/...
2,2025/298,Statutory Correspondence - Dormansland Parish ...,https://plandocs.tandridge.gov.uk/my-requests/...


In [4]:
import pandas as pd
import requests
from pathlib import Path
from pdfminer.high_level import extract_text
from urllib.parse import urlparse, parse_qs
import tempfile
import os
from typing import Optional, Tuple
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed


def normalize_tandridge_pdf_url(u: str) -> Tuple[str, Optional[str]]:
    """
    Convert various Tandridge 'viewer' links into a direct PDF handler URL.
    Returns (direct_pdf_url, doc_id or None).
    Handles:
      - .../my-requests/document-viewer?DocNo=25362793
      - ...docno=25362793 anywhere
      - already-direct handler links
    """
    try:
        parsed = urlparse(u)
        qs = parse_qs(parsed.query)
        # Catch both DocNo and docno
        docno = None
        for key in ("DocNo", "docno"):
            if key in qs and len(qs[key]) > 0:
                docno = qs[key][0]
                break

        if docno:
            direct = (
                "https://plandocs.tandridge.gov.uk/"
                "w2webparts/Resource/Civica/Handler.ashx/Doc/pagestream"
                f"?cd=inline&pdf=true&docno={docno}"
            )
            return direct, docno

        # If path already looks like the handler with docno in path or query, leave as-is
        if "Handler.ashx" in u and ("docno=" in u.lower() or "DocNo=" in u):
            return u, None

        # If it ends with .pdf, also fine
        if parsed.path.lower().endswith(".pdf"):
            return u, None

        # Fallback: return original
        return u, None
    except Exception:
        return u, None


def build_retrying_session(max_retries: int = 3, backoff_factor: float = 1.0) -> requests.Session:
    session = requests.Session()
    retry_strategy = Retry(
        total=max_retries,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"],
        backoff_factor=backoff_factor,
        raise_on_status=False,
        respect_retry_after_header=True,
    )
    adapter = HTTPAdapter(max_retries=retry_strategy, pool_connections=20, pool_maxsize=20)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session


def fetch_pdf_bytes(session: requests.Session, url: str, timeout_connect: int = 30, timeout_read: int = 90) -> Tuple[bytes, int, str]:
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "application/pdf,application/octet-stream,*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        # Some Civica backends like having a same-site referer
        "Referer": "https://plandocs.tandridge.gov.uk/",
    }
    with session.get(url, headers=headers, timeout=(timeout_connect, timeout_read), stream=True) as r:
        r.raise_for_status()
        content_type = r.headers.get("content-type", "")
        total = int(r.headers.get("content-length", 0)) if r.headers.get("content-length") else 0
        buf = bytearray()
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                buf.extend(chunk)
        return bytes(buf), (len(buf) if total == 0 else total), content_type


def extract_pdf_text_from_bytes(pdf_bytes: bytes) -> str:
    # pdfminer works from a file; use a temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        tmp.write(pdf_bytes)
        tmp_path = tmp.name
    try:
        text = extract_text(tmp_path) or ""
        # Keep line breaks, trim trailing spaces
        text = "\n".join(line.rstrip() for line in text.splitlines())
        return text
    finally:
        try:
            os.unlink(tmp_path)
        except OSError:
            pass


def process_one_row(row: pd.Series,
                    session: requests.Session,
                    connect_timeout: int = 30,
                    read_timeout: int = 120) -> dict:
    app_no = row.get("application_number", "")
    file_name = row.get("file_name", "")
    url = row.get("url", "")

    # Normalize to direct PDF when possible
    normalized_url, doc_id = normalize_tandridge_pdf_url(url)

    result = {
        "application_number": app_no,
        "file_name": file_name,
        "url": url,
        "normalized_url": normalized_url,
        "doc_id": doc_id,
        "content": "",
        "text_len": 0,
        "bytes_downloaded": 0,
        "status": "ok",
        "error": ""
    }

    try:
        pdf_bytes, sz, ctype = fetch_pdf_bytes(session, normalized_url, timeout_connect=connect_timeout, timeout_read=read_timeout)
        result["bytes_downloaded"] = len(pdf_bytes)
        # Quick sanity: PDF header
        if not pdf_bytes.startswith(b"%PDF"):
            # Not all servers include %PDF at byte 0 (rare), but warn if so
            # We'll still try to parse; pdfminer may still handle it if it is a PDF.
            pass
        text = extract_pdf_text_from_bytes(pdf_bytes)
        result["content"] = text
        result["text_len"] = len(text)
        # If text is empty, mark but don't fail (could be scanned PDF)
        if result["text_len"] == 0:
            result["status"] = "empty_text"
    except requests.RequestException as e:
        result["status"] = "download_error"
        result["error"] = str(e)
    except Exception as e:
        result["status"] = "extract_error"
        result["error"] = str(e)

    return result


def extract_all_files_content(
    files_df: pd.DataFrame,
    max_workers: int = 6,
    connect_timeout: int = 30,
    read_timeout: int = 120,
    max_retries: int = 4,
    backoff_factor: float = 1.0,
) -> pd.DataFrame:
    """
    Process every row in `files_df` (expects columns: application_number, file_name, url).
    Returns a DataFrame with content + metadata per row.
    """
    required_cols = {"application_number", "file_name", "url"}
    missing = required_cols - set(files_df.columns)
    if missing:
        raise ValueError(f"files_df is missing required columns: {missing}")

    session = build_retrying_session(max_retries=max_retries, backoff_factor=backoff_factor)

    results = []
    if max_workers and max_workers > 1:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futures = {
                ex.submit(process_one_row, row, session, connect_timeout, read_timeout): idx
                for idx, row in files_df.iterrows()
            }
            for fut in as_completed(futures):
                results.append(fut.result())
    else:
        for _, row in files_df.iterrows():
            results.append(process_one_row(row, session, connect_timeout, read_timeout))

    # Close session
    try:
        session.close()
    except Exception:
        pass

    out_df = pd.DataFrame(results)
    return out_df


results_df = extract_all_files_content(df, max_workers=6)

# results_df.to_csv("documentContent.csv", index=False)

In [5]:
results_df

,application_number,file_name,url,normalized_url,doc_id,content,text_len,bytes_downloaded,status,error
0,2025/298,Decision Notice - Decision notice 2025-298,https://plandocs.tandridge.gov.uk/my-requests/...,https://plandocs.tandridge.gov.uk/w2webparts/R...,25324970,Tandridge District Council\nTown & Country Pla...,2810,175293,ok,
1,2025/298,Officer Report - Officer report-2025-298,https://plandocs.tandridge.gov.uk/my-requests/...,https://plandocs.tandridge.gov.uk/w2webparts/R...,25324968,Officer Report: Lawful Development Certificate...,9746,185681,ok,
2,2025/298,Statutory Correspondence - Dormansland Parish ...,https://plandocs.tandridge.gov.uk/my-requests/...,https://plandocs.tandridge.gov.uk/w2webparts/R...,25313421,From:\nSent:\nTo:\nSubject:\nAttachments:\n\np...,3281,306344,ok,


In [10]:
contentDF = results_df.drop(columns=["normalized_url", "doc_id", "text_len", "bytes_downloaded", "status", "error"], errors="ignore")


In [11]:
contentDF

,application_number,file_name,url,content
0,2025/298,Decision Notice - Decision notice 2025-298,https://plandocs.tandridge.gov.uk/my-requests/...,Tandridge District Council\nTown & Country Pla...
1,2025/298,Officer Report - Officer report-2025-298,https://plandocs.tandridge.gov.uk/my-requests/...,Officer Report: Lawful Development Certificate...
2,2025/298,Statutory Correspondence - Dormansland Parish ...,https://plandocs.tandridge.gov.uk/my-requests/...,From:\nSent:\nTo:\nSubject:\nAttachments:\n\np...
